# JijModeling 1.10.0 リリースノート

JijModeling 1.10.0 は、ライブラリの安定性と利便性を向上させる重要なバグ修正と機能強化を含みます。

## 機能強化

### `Interpreter` における one-hot 制約の検出

`Interpreter` は、制約評価の仮定で次の形式の one-hot 制約を検出するようになりました:

$$
\sum_{i=1}^{n} x_i = 1, \quad x_i \in \{0, 1\}
$$

この機能追加により、ソルバーのアダプタ実装時に利用可能な制約ヒントが提供されます。検出された制約ヒントは `ommx.v1.Instance` の `constraint_hints` プロパティからアクセスできます。

In [ ]:
import jijmodeling as jm

n = jm.Placeholder("n")
x = jm.BinaryVar("x", shape=(n,))
i = jm.Element("i", belong_to=(0,n))

problem = jm.Problem("problem")
# one-hot 制約を追加
problem += jm.Constraint("onehot", jm.sum(i, x[i]) == 1)

interpreter = jm.Interpreter({"n": 3})
instance = interpreter.eval_problem(problem)

# 検出された制約ヒントの情報を表示
print(instance.raw.constraint_hints)

将来のアップデートでは、SOS1/SOS2 や n-hot 制約を含むさまざまな制約の検出が可能になる予定です。

### 型ヒントの更なる改善

Pythonサブモジュールを跨いだ型ヒントが大幅に改善され、これまで Pyright などで発生していた型エラーが解消されました。この改善は `pyo3-stub-gen` での機能改善（[参考リンク](https://github.com/Jij-Inc/pyo3-stub-gen/releases/tag/0.6.1)）に立脚しており、サブモジュールに対してもシームレスに型検査が行われるようになりました。

以前は、以下のようなサブモジュールへの呼び出しで型エラーが頻繁に発生していました:

![image.png](./assets/jijmodeling-1.10.0_01.png)

本リリースでこの問題は解消し、より快適な開発体験が得られるようになります：

![image.png](./assets/jijmodeling-1.10.0_02.png)

## バグ修正

### Jagged Array に対する `ndim` 計算の修正

このリリースでは、Jagged Arrayの次元 (`ndim`) 計算方法を修正しました。このバグは `Interpreter` の初期化中にエラーを引き起こし、MIPLIB から読み込まれたインスタンスの評価を妨げていました。

In [ ]:
import jijmodeling as jm

# MIPLIB インスタンス "air05" をロード
dataset = jm.dataset.Miplib()
problem, instance_data = dataset.load("air05")

try:
    # jijmodeling <= 1.9.0 では、これらのいずれかの行が InterpreterError を引き起こします
    interpreter = jm.Interpreter(instance_data)
    interpreter.eval_problem(problem)
except jm.InterpreterError as e:
    print(f"InterpreterError: {e}")

### 目的関数の算術代入演算子 (`+=`) の挙動を修正

目的関数に対する `+=` 演算子に関する長期に渡ったバグを修正しました。これまで目的関数に対する `+=` は目的関数を置き換えてしまっていましたが、本バージョン以降は新たに項が加算されます。

In [ ]:
import jijmodeling as jm

x = jm.BinaryVar("x")
y = jm.BinaryVar("y")

problem = jm.Problem("problem")
# x を目的関数に加算
problem += x
# y を目的関数に加算
problem += y

# jijmodeling <= 1.9.0 では x + y ではなく y になってしまっていた
assert jm.is_same(problem.objective, x + y)

## 変更履歴

### バグ修正

- Jagged Array の `ndim` 計算を修正し、Interpreter で MIPLIB 形式の評価をシームレスに行えるようになりました。
- 目的関数に対する `+=` 演算子が既存の目的関数に項を適切に追加するように修正しました。

### 機能強化

- One-Hot 制約検出：次の形式の制約を one-hot 制約として検出できるようになりました：
    $
      \displaystyle \sum_{i=1}^{n} x_i = 1, \quad x_i \in \{0, 1\}
    $
  - 左辺: 正確に1つの総和記号。
  - 総和の被演算子: インデックス付きの単一のバイナリ変数。
  - 制約タイプ: 等式。
  - 右辺: 定数 `1`。
- `pyo3-stub-gen` の改善を通じて型ヒントサポートを向上させ、サブモジュールに対する Pyright エラーを解消しました。